# Importing Fake New Classifier Dataset 

In [1]:
import pandas as pd ## Importing pandas library for data manipulating and reading

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
## Shape of the dataset ---> no. of records is 20800 
df.shape

(20800, 5)

In [4]:
## Handling the Nan values
# Droping the recored having Nan values
df.dropna(inplace=True)

In [5]:
df.shape

(18285, 5)

In [6]:
X = df.drop('label',axis =1)
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
## Importing Tensorflow version > 2.0
# Importing Embedding for creating Embedding Layers
# Importing pad_sequences for making the no. of words in a sentence Equal
# Importing one_hot to make one hot representation of the words in the sentences
# Importing LSTM model 
# Importing Dense layer for the output

In [10]:
import tensorflow as tf 

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
## Vocabulary Size 
voc_size = 5000

In [13]:
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

# Data Preprocessing

In [16]:
import nltk # nltk library used for test analysis
import re # re for regular expression
from nltk.corpus import stopwords # stopwords to remove the unnecessary words

In [17]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [19]:
## Step - 1 
# One-Hot representation

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[4040, 3026, 3754, 4438, 2696, 297, 503]

In [21]:
## Step - 2 
# Creating Embedding representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2634 2565 1319]
 [   0    0    0 ... 2696  297  503]
 [   0    0    0 ... 4859 3964 2480]
 ...
 [   0    0    0 ... 2068  396  915]
 [   0    0    0 ... 3847 1834 2111]
 [   0    0    0 ... 4325 4286   26]]


In [23]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4040, 3026, 3754, 4438, 2696,  297,  503])

# Embedding Layer and Model Creation 

In [24]:
## Creating Model
embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
## Spliting the data into train and test set 

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 30ms/step - loss: 0.3396 - accuracy: 0.8367 - val_loss: 0.2012 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1406 - accuracy: 0.9433 - val_loss: 0.2006 - val_accuracy: 0.9173
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0993 - accuracy: 0.9618 - val_loss: 0.2330 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0697 - accuracy: 0.9741 - val_loss: 0.2564 - val_accuracy: 0.9051
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0477 - accuracy: 0.9849 - val_loss: 0.2946 - val_accuracy: 0.9153
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0320 - accuracy: 0.9894 - val_loss: 0.3361 - val_accuracy: 0.9097
Epoch 7/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0156 - accuracy: 0.9954 - val_loss: 0.4912 - val_accuracy: 0.9057

In [31]:
## Prediction using LSTM RNN Neural Network
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [32]:
## Creating a Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[3109,  310],
       [ 215, 2401]], dtype=int64)

In [33]:
# Evaluating the Testing Accuracy Score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9130074565037283

# Hyperparameter Tuning using Dropout Layer

In [34]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [36]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 19ms/step - loss: 0.3548 - accuracy: 0.8291 - val_loss: 0.2173 - val_accuracy: 0.9087
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1513 - accuracy: 0.9408 - val_loss: 0.2031 - val_accuracy: 0.9203
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.1116 - accuracy: 0.9564 - val_loss: 0.2173 - val_accuracy: 0.9221
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0865 - accuracy: 0.9681 - val_loss: 0.2205 - val_accuracy: 0.9138
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0666 - accuracy: 0.9767 - val_loss: 0.2780 - val_accuracy: 0.9165
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0509 - accuracy: 0.9820 - val_loss: 0.2860 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0347 - accuracy: 0.9880 - val_loss: 0.3474 - val_accuracy: 0.9155

In [37]:
y_pred=model.predict_classes(X_test)

In [38]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[3109,  310],
       [ 196, 2420]], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9161557580778791